# Benchmarking pLMs for runtime efficiency

In [4]:
# Common Imports
from transformers import AutoModel
import torch
import json
import time

In [ ]:
# Benchmarking Function

# Set trials
min_batch_size = 8
max_batch_size = 32
inc_batch_size = 8

min_sequence_length = 64
max_sequence_length = 512
inc_sequence_length = 64

iterations = 20

# Load to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_name = torch.cuda.get_device_name(device)

model_results = []
def benchmark(model, model_name):
    with torch.no_grad():
        print((' Benchmarking using ' + device_name + ' ').center(80, '*'))
        print(' Start '.center(80, '*'))
        for sequence_length in range(min_sequence_length,max_sequence_length+1,inc_sequence_length):
            for batch_size in range(min_batch_size,max_batch_size+1,inc_batch_size):
                start = time.time()
                for i in range(iterations):
                    input_ids = torch.randint(1, 20, (batch_size,sequence_length)).to(device)
                    model(input_ids=input_ids)
                end = time.time()
                ms_per_protein = (end-start)/(iterations*batch_size)
                print('Sequence Length: %4d \t Batch Size: %4d \t Ms per protein %4.2f' %(sequence_length,batch_size,ms_per_protein))
                curr = {
                    "Sequence Length":   sequence_length,
                    "Batch Size":    batch_size,
                    "Ms per protein": ms_per_protein
                }
                model_results.append(curr)

                # save the results to json
                with open(f'./{model_name}_results.json', 'w') as f:
                    json.dump(model_results, f)

            print(' Done '.center(80, '*'))
        print(' Finished '.center(80, '*'))
        

## Benchmarking ProtT5-XLUniref-50

Original model to benchmark embedding creation runtime against.

In [ ]:
# Model Import
from transformers import T5EncoderModel

# Load the model
t5xl_model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device)
t5xl_model = t5xl_model.to(device)
t5xl_model.eval()

benchmark(t5xl_model, "T5XL")

# Benchmark Ankh

Updated model from Elneggar Lab, follow-up to ProtTrans

In [ ]:
import ankh

# Load base model
ankh_model = ankh.load_base_model()
ankh_model = ankh_model[0].to(device)
ankh_model.eval()

benchmark(ankh_model, "Ankh")

# Benchmark Albert

In [ ]:
albert_model = AutoModel.from_pretrained("Rostlab/prot_albert")
albert_model = albert_model.to(device)
albert_model.eval()

benchmark(albert_model, "Albert")

# Benchmark XLNet

In [ ]:
xlnet_model = AutoModel.from_pretrained("Rostlab/prot_xlnet")
xlnet_model.to(device)
xlnet_model.eval()

benchmark(xlnet_model, "XLNet")

# Benchmark FAesm

In [ ]:
from faesm.esm import FAEsmForMaskedLM

faesm_model = FAEsmForMaskedLM.from_pretrained("facebook/esm2_t33_650M_UR50D")
faesm_model.to(device)
faesm_model.eval().to(torch.float16)

benchmark(faesm_model, "FAEsm")

In [ ]:
from lobster.model import LobsterPMLM
lobster_model = LobsterPMLM("/scratch/akeluska/qiyun-thesis/models/lobster_24M")
lobster_model.eval().to(device).to(torch.float32)

benchmark(lobster_model, "Lobster_24M")

In [ ]:
lobster_150M_model = LobsterPMLM("/scratch/akeluska/qiyun-thesis/models/lobster_150M")
lobster_150M_model.eval().to(device).to(torch.float32)

benchmark(lobster_150M_model, "Lobster_150M")